# Open cast mining

An opencast uranium mine is being prospected.  Based on the results of
some test drillings the mine has been subdivided into exploitation units
called blocks.  The pit needs to be terraced to allow the trucks to drive
down to its bottom.  The uranium deposit extends from east to west.
The pit is limited in the west by a village and in the east by a group of
mountains.  Taking into account these constraints, 18 blocks of 10,000
tonnes on three levels have been identified:

|Level   |c1|c2|c3|c4  |c5  |c6  |c7 |c8|
|--------|--|--|--|----|----|----|---|--|
|Level 1 |1 |2 |3 |4   |5   |6   |7  |8 |
|Level 2 |  |9 |10|11  |12  |13  |14 |  |
|Level 3 |  |  |15|16  |17  |18  |   |  |

To extract a block, three blocks of the level above it need to be extracted:
the block immediately on top of it, and also due to the constraints on the
slope, the blocks to the right and the left.

It costs \\$100 per tonne to extract a block of level 1, \\$200 per tonne for
a block of level 2, and \\$300 per tonne for a block of level 3, with the
exception of blocks 9, 14, 15, 16 and 18, that are formed of a very hard rock
rich in quartz and cost \\$1000 per tonne.  The only blocks that contain
uranium are 1, 7, 10, 12, 17 and 18.
Their market value is 200, 300, 500, 200, 1000 and \\$1200/tonne respectively.

### Which blocks should be extracted to maximize the total benefit?

In [1]:
# Load the gams extension
%load_ext gams_magic

In [2]:
%%gams
set c /c1*c8/;
set lev /1*3/;

table block(lev,c)
    c1 c2 c3 c4 c5 c6 c7 c8
1   1  2  3  4  5  6  7  8
2      9 10 11 12 13 14
3        15 16 17 18;

table cost(lev,c)
   c1   c2   c3   c4   c5   c6   c7   c8
1  100  100  100  100  100  100  100  100
2      1000  200  200  200  200 1000 
3           1000 1000 1000  300 1000;

table p(lev,c)
   c1   c2   c3   c4   c5   c6   c7   c8
1  200                           300
2            500       200
3                     1000 1200;

binary variable extract(lev,c);
variable z;

equations extractdef, obj;

extractdef(lev,c)$(ord(lev)>1).. 3*extract(lev,c) =L= extract(lev-1,c-1) + extract(lev-1,c) + extract(lev-1,c+1);

obj.. z =E= sum((lev,c), extract(lev,c) * ( p(lev,c) - cost(lev,c)   )     );

model mining /all/;
solve mining using mip maximizing z; 
display extract.L, z.L;


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),300.0,17,25,MIP,CPLEX,1.344


In [3]:
%gams_pull -d extract block
import pandas as pd 

block.columns = ['lev', 'c', 'block']
soln=extract.loc[extract['level'] > 0]
soln = pd.merge(soln, block, left_on=['lev','c'], right_on=['lev','c'])

soln.set_index('block',inplace=True)
print(soln[['level']])


       level
block       
1.0      1.0
7.0      1.0
